# Beleglösung SFI

![Aufbau](img/aufbau.png)

- Parameter des Wärmeübertragers:
  - Rohrdurchmesser: $d_r = 15 mm$ 
  - kinetische Viskosität Luft: $\nu = 1.5 \cdot 10^{-5}  \frac{m^2}{s}$
  - Dichte Luft: $\rho = 1.2 \frac{kg}{m^3}$
  - Anströmgeschwindigkeit: $U_b = 1 \frac{m}{s}$
  - Rohrabstand x-Richtung: $s_x = b \cdot d_r = 15 mm$
  - Rohrabstand y-Richtung: $s_y = a \cdot d_r = 45 mm$

In [2]:
d_r = 15e-3  # Rohrdurchmesser in m
nu_L = 1.5e-5  # kinetische Viskosität Luft in m^2/s
rho_L = 1.2    # Dichte Luft in kg/m^3
U_b = 1.0      # Anströmgeschwindigkeit in m/s
s_x  = 15e-3  # Rohrabstand x-Richtung in m
s_y  = 45e-3  # Rohrabstand y-Richtung in m

# Aufgabe 1: Vorüberlegungen

a) Schätzen Sie die Strömungsgeschwindigkeit Ue im engsten Querschnitt des Wärmeübertragers ab.

- Größter Querschnitt $A_{ein} = a \cdot d_r \cdot z$
- Engster Querschnitt $A_{eng} = (a \cdot d_r - 2d_r) \cdot z = (a - 2) \cdot d_r * z$

- Kontigleichung:

$$A_{ein} \cdot U_b = A_{eng} \cdot U_e$$
$$\Rightarrow U_e = \frac{A_{ein}}{A_{eng}} \cdot U_b $$

In [3]:
A_in = s_y # m²
A_eng = s_y - 2 * d_r # m²

U_e = U_b * A_in / A_eng
print(f"Strömungsgeschwindigkeit Ue im engsten Querschnitt: {U_e:.2f} m/s")


Strömungsgeschwindigkeit Ue im engsten Querschnitt: 3.00 m/s


b) Bestimmen Sie den Strömungscharakter (laminar/turbulent) mithilfe der Reynoldszahl $Re= \frac{U_b \cdot d_r}{\nu}$. Bis zu einer Reynoldszahl von 100 ist die Strömung laminar. Ab einer Reynoldszahl von 10000 ist die Strömung voll turbulent. 

In [4]:
Re = U_e * d_r / nu_L
print("Reynoldszahl:", Re)

if Re < 100:
    print("Die Strömung ist laminar.")
elif Re > 10000:
    print("Die Strömung ist turbulent.")
else:
    print("Die Strömung ist im Übergangsbereich zwischen laminar und turbulent.")


Reynoldszahl: 3000.0
Die Strömung ist im Übergangsbereich zwischen laminar und turbulent.


c) Wird für die Simulation ein Turbulenzmodell benötigt? Falls ja, benutzen Sie z.B. das kEpsilon-Modell.

- Ja, da die Strömung nicht laminar ist und zum Teil turbulent.

# Aufgabe 2: Vernetzung im Unterordner "meshing"


a) Analysieren Sie die gegebene .stl-Geometrie in „constant/triSurface/“.
Ermitteln Sie die Abmessungen des Gebiets (xmin, ymin, zmin) (xmax, ymax, zmax).
Erstellen Sie eine Skizze des Gebiets und geben Sie darin die Namen der
Teilflächen aus der .stl-Datei an.

- surfaceCheck *.stl
- (xmin, ymin, zmin) = (-0.1425 -0.0225 -0.002) 
- (xmax, ymax, zmax) = (0.5175 0.0225 0.002)

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('static')


# STL laden
mesh = pv.read("./wärmeübertrager/meshing/constant/triSurface/domain_a3_b1.stl")

# 3D-Plot
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, color="lightgray")
pl.add_axes()  # Koordinatenachsen anzeigen

# Kameraposition einstellen
pl.camera_position = 'xy'
cam, foc, up = pl.camera_position
dx = -0.2
pl.camera_position = [(cam[0]+dx, cam[1], cam[2]), (foc[0]+dx, foc[1], foc[2]), (up[0], up[1], up[2])]  # Kameraposition einstellen
pl.camera.zoom(4)

# Domain
x_min, x_max = -0.1, 0.08
y_min, y_max = -0.0225, 0.0225
z = 0.01
pointa = [x_min, y_min, z]
pointb = [x_min, y_max, z]
pointc = [x_max, y_min, z]
domain = pv.Rectangle([pointa, pointb, pointc])
pl.add_mesh(domain, color="green", show_edges=True, line_width=5, render_lines_as_tubes=False, opacity=0.2)

# Inlet
dy = 0.01
dx = -0.01
inlet = pv.Line((x_min, y_min -  dy, z), (x_min, y_max + dy, z))
pl.add_mesh(inlet, color="black", line_width=5)
label_inlet = pv.Label("Inlet", position=(x_min + dx, y_max + 0.015, z), size=40)
pl.add_actor(label_inlet)
label_inlet_x = pv.Label(f"x = {x_min}", position=(x_min + dx, y_min - 0.02, z), size=30)
pl.add_actor(label_inlet_x)

# Outlet
outlet = pv.Line((x_max, y_min -  dy, z), (x_max, y_max + dy, z))
pl.add_mesh(outlet, color="black", line_width=5)
label_outlet = pv.Label("Outlet", position=(x_max + dx, y_max + 0.015, z), size=40)
pl.add_actor(label_outlet)
label_outlet_x = pv.Label(f"x = {x_max}", position=(x_max + dx, y_min - 0.02, z), size=30)
pl.add_actor(label_outlet_x)

# Top und Bottom
top_label_y = pv.Label(f"y_max = {y_max}", position=((x_min + x_max)/2 + dx, y_max + 0.015, z), size=30)
bottom_label_y = pv.Label(f"y_min = {y_min}", position=((x_min + x_max)/2 + dx, y_min - 0.02, z), size=30)
pl.add_actor(top_label_y)
pl.add_actor(bottom_label_y)

# Gitter und Achsenbeschriftung anzeigen
pl.show()


/home/anh/Documents/studium/venv/lib/python3.12/site-packages/trame/ui/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


Widget(value='<iframe src="http://localhost:35085/index.html?ui=P_0x756fb644a8d0_0&reconnect=auto" class="pyvi…

b & c & d & e) Ergebnis des Meshings in Paraview.

![SnappyHexMesh](img/aufgabe2_snappyHexMesh.png)

![SnappyHexMeshDomain](img/aufgabe2_snappyHexMeshDomain.png)

# Aufgabe 3 Isotherme Berechnung im Hauptordner


a) Stellen Sie die Randbedingungen der Felder im Ordner „0“ ein.

- Anströmgeschwindigkeit: $U_b = 1 \frac{m}{s}$
- Turbulenzintensität: $I = 10\%$
- turbulente Längenskale $L = \frac{d_r}{2} = 7,5mm$

![Anströmgeschwindigkeit](img/aufgabe3_RB_Ub.png)

![Turbulenzintensität](img/aufgabe3_RB_I.png)

![Turbulente Längenskale](img/aufgabe3_RB_L.png)

b) Integrale Strömungsgrößen sollen mit functionObjects berechnet werden,
editieren Sie die Dateien „avgPres“ und „forcesPipe“ damit der mittlere Druck
am Einlass, sowie die Kraft auf die Rohre des Wärmeübertragers berechnet
werden.

![avgPres](img/aufgabe3_avgPres.png)

c) Bearbeiten Sie die verbleibenden ToDos und führen Sie die stationäre
Simulation mit foamRun durch.

d) Stellen Sie die Konvergenz der Lösung sicher. Stellen Sie dafür den Verlauf
über den Iterationen von Residuen, mittlerem Druck am Einlass und der
Gesamtkraft auf die Rohrwände grafisch dar.

![Residuen](img/aufgabe3_residuals.png)

![Mittlerer Druck](img/aufgabe3_avgPres_inlet.png)

![Kraft auf Rohrwände](img/aufgabe3_maxForce.png)

# Aufgabe 4


# Aufgabe 5
